<a href="https://colab.research.google.com/github/ChoHadam/Insta_User_Profiling/blob/master/Data_Analysis/Classification_KOBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x65d8e000 @  0x7f451eed5615 0x591f47 0x4cc229 0x4cc38b 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x509918 0x50a64d
     |████████████████████████████████| 846.0MB 21kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 4.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588528 sha256=b198d6e961504230753cd44e33bc15f68d0b5119afb1a688ac745835cce5289e
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 4.9MB

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git

  Cloning https://****@github.com/SKTBrain/KoBERT.git to /tmp/pip-req-build-jl7kmcyr
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-jl7kmcyr
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12825 sha256=ea807f91eb15adf47800dd894da68d78326a6380f8f8efd42e086e726c991608
  Stored in directory: /tmp/pip-ephem-wheel-cache-10fyngjr/wheels/43/7b/50/91bc4d7bebb30b62f31a30a537787eb512f27506909162ba14
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

In [4]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
#GPU 사용 시
device = torch.device("cuda:0")

# **SKT 제공 KoBert 이용**

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [7]:
train_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/insta_profiling1/Data_Preprocessing/Data_Set/train_data.csv'
test_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/insta_profiling1/Data_Preprocessing/Data_Set/test_data.csv'

In [8]:
train_data = pd.read_csv(train_path, engine='python', encoding='utf-8', index_col=0)
test_data = pd.read_csv(test_path, engine='python', encoding='utf-8', index_col=0)
len(train_data), len(test_data)

(1051084, 262993)

In [9]:
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.drop(['index'], axis=1)
test_data = test_data.dropna()
test_data = test_data.reset_index()
test_data = test_data.drop(['index'], axis=1)
len(train_data), len(test_data)

(1030227, 257422)

In [10]:
dataset_train = []
for i in tqdm(range(len(train_data))):
    dataset_train.append([train_data['content'][i], int(train_data['label'][i])])
dataset_train[:5]

100%|██████████| 1030227/1030227 [00:45<00:00, 22776.12it/s]


[['#슈봉에서 몰 먹어도 맛남😋 유치원 단체 러블리 고객님^^ 퇴장 후~~ #당 충전~~ #단호박파운드케익🎃 #치즈샌드마카롱 #유치원단체환영 #유치원단체문의#좋아요👍 #슈슈봉봉위례신도시점#위례중앙타워 키카#프리미엄키즈카페#송파맘#위례맘#성남맘#하남맘#도곡맘#강남맘#대치동맘#키즈카페레스토랑#키즈카페맛집#디져트는모후모후',
  10],
 ['머리를 1년반?만에 염색했어요ㅋ 자르고 염색하고 영양도 #돈좀쓴날 #망고헤어 #염색', 10],
 ['로아앤 제인 팝업 스토어 ~ 오늘이 마지막이라 스케쥴 많은데 부랴부랴~~^^ 포토존에서 사진 한장 찍을래두 부산스러운 현서~😂😂 다른분들은 예쁘게 사진두 찍어오고 했드만 우린....😢😢 저번엔 못뵀던 로아파,제인맘님 ~로아 이쁜게 엄마,아빠,이모까지 골고루 닮아서인가보다~~인상도 좋으시고 현서 알아봐주시고 쇼핑 하러 간거지만 얘기 더 하고 싶어도 현서가 도움을 안주니~~그래두 즐건 쇼핑이었어요~~^^ - - #애스타그램  #육스타그램  #둘째사랑  #데일리 #자매스타그램  #로아앤제인 현대 무역센터 팝업스토어  #로아 패밀리👍👍',
  10],
 ['#일요일 #점심메뉴  더운 날씨에는 역시 냉면💁🏻 그러나 냉면에는 역시 갈비👀 (냉면에는 돼지갈비가 짱이지👊🏻👊🏻👊🏻)', 30],
 ['#서울어딘가 #연남동 #반디스튜디오 #기타치고싶은 유나 #피노라벨 #초등학생 #주니어모델 #유나 #여름신상 #인연이란 #그냥 내버려두어도 저절로 자라는 야생초가 아니라 #인내를 가지고 공과 시간을 들여야  #비로소 향기로운 꽃을 피우는 한포기 난초인 것이다#한비야  #바람의 딸 우리땅에 서다',
  10]]

In [11]:
dataset_test = []
for i in tqdm(range(len(test_data))):
    dataset_test.append([test_data['content'][i], int(test_data['label'][i])])
dataset_test[:5]

100%|██████████| 257422/257422 [00:07<00:00, 36538.48it/s]


[['엄마~ 오늘 재하 생일이지? 엄마~ 오늘 재야 생일이디~? 엄마~ 엄ㅁㅏ~  #그래 #너오늘생일이다', 10],
 ['띠로리', 10],
 ['Look Im serious', 10],
 ['사랑스런 노랑궁뎅이와 짧은두다리 잉!♡', 10],
 ['요즘 나랑 절교 직전인 우리 다해😅😅 아침에 너무 말을 안들어서 혼냈더니 비밀편지라고 쓰윽 갔다준당💦  (번역) 엄마 사랑하고 미안 화풀어 이젠 안그럴게(추측ㅋㅋ)  너가 자꾸 이러니까 절교가 안되자나💕💕 이렇게 오늘도 난 지고말았음🙄🙄🙄🙄🙄🙄  #나머지글은아무말대잔치에요 #해석불가ㅋㅋ #딸스타그램 #5살 #둘째 #애교쟁이 #세종맘',
  10]]

In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [14]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [15]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [16]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

RuntimeError: ignored

In [28]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [20]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [21]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [22]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [23]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [25]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        print(loss)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


RuntimeError: ignored